#### Initialization

In [1]:
#pip install scipy

In [2]:
import pandas as pd
import csv
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
ids_A = ["A8", "A10", "A11", "A12"]
ids_B = ["B4", "B5", "B6", "B7"]

Read experiment csv files into dictionarys

In [4]:
a_sample_dict, b_sample_dict, a_result_dict, b_result_dict = {}, {}, {}, {}
tmp = []

for id in ids_A:
    for i in range(125):
        tmp.append(pd.read_csv(fr"C:\Users\kevin\Desktop\ExperimentResults\Group_A\userID_{id}\SampledMetrics\trialIteration_{i}_samples.csv", sep=";"))
        #tmp[i].dropna(axis='columns')
    a_sample_dict[id] = tmp
    tmp = []
    
    #fixing broken first row in results 
    result_path = fr'C:\Users\kevin\Desktop\ExperimentResults\Group_A\userID_{id}'
    with open(fr"{result_path}\Result.csv", 'rt') as inp, open(fr"{result_path}\Result_edit.csv", 'wt') as out:
        writer = csv.writer(out)
        for row in csv.reader(inp):
            if row != ['sep=;']:
                writer.writerow(row)

    a_result_dict[id] = pd.read_csv(fr"{result_path}\Result_edit.csv", sep=";")
    # dropping irrelevant columns
    # a_result_dict[id].drop('TrialIteration', axis=1, inplace=True) # might be interesting, when sorting the columns after trialID
    a_result_dict[id].dropna(axis='columns', inplace=True)
    #a_result_dict[id] = a_result_dict[id].sort_values(by=['trialID'])

for id in ids_B:
    for i in range(125):
        tmp.append(pd.read_csv(fr"C:\Users\kevin\Desktop\ExperimentResults\Group_B\userID_{id}\SampledMetrics\trialIteration_{i}_samples.csv", sep=";"))
        tmp[i].dropna(axis='columns')
        tmp[i].drop(columns=['g_r', 'injected_rotations', 'additional_virtual_rotation_accumulation',
                          'virtual_position', 'virtual_direction', 'virtual_euler', 'virtual_rotation'],
                    inplace=True)
    b_sample_dict[id] = tmp
    tmp = []

    #fixing broken first row in results 
    result_path = fr'C:\Users\kevin\Desktop\ExperimentResults\Group_B\userID_{id}'
    with open(fr"{result_path}\Result.csv", 'rt') as inp, open(fr"{result_path}\Result_edit.csv", 'wt') as out:
        writer = csv.writer(out)
        for row in csv.reader(inp):
            if row != ['sep=;']:
                writer.writerow(row)#
                
    b_result_dict[id] = pd.read_csv(fr"{result_path}\Result_edit.csv", sep=";")
    # dropping irrelevant columns
    b_result_dict[id].drop(columns=['min_g_r', 'max_g_r', 'g_r_average', 'sum_injected_rotation_g_r(IN DEGREES)', 'injected_rotation_average',
                                    'real_dir_at_virt_yaw', 'virt_dir_at_virt_yaw', 'max_yaw_virt', 'real_pos_at_virt_yaw', 'virt_pos_at_virt_yaw',
                                    'real_euler_at_real_yaw','virt_euler_at_real_yaw', 'real_euler_at_virt_yaw', 'virt_euler_at_virt_yaw',
                                    'real_rot_at_real_yaw',	'virt_rot_at_real_yaw', 'real_rot_at_virt_yaw' ,'virt_rot_at_virt_yaw'],
                           inplace=True)
    # b_result_dict[id].drop('TrialIteration', axis=1, inplace=True)  # might be interesting, when sorting the columns after trialID
    b_result_dict[id].dropna(axis='columns', inplace=True)
    #b_result_dict[id] = b_result_dict[id].sort_values(by=['trialID'])


#a_result_dict['A10']
#b_result_dict['B4']
#a_sample_dict['A10'][25]
a_result_dict['A10'].loc[:, 'min_g_r':'injected_rotation_average']

,min_g_r,max_g_r,g_r_average,sum_injected_rotation_g_r(IN DEGREES),injected_rotation_average
0,-2.721025,1.972275,0.004942,9.039402,0.000963
1,-5.269212,5.383467,0.009485,19.448920,0.001429
2,-4.990340,4.139502,0.011336,18.598520,0.002283
3,-2.886984,2.531294,0.006964,15.505280,0.001890
4,-3.344656,7.295310,0.009690,19.751630,0.002661
...,...,...,...,...,...
121,-3.839200,7.484250,0.008956,14.342270,0.001819
122,-1.816806,6.931334,0.011709,21.298190,0.003384
123,-1.490812,1.948283,0.006214,17.180910,0.001340
124,-3.028456,13.686770,0.008138,36.969550,0.002977


read questionnaire csv and concat to result csv

In [5]:
questionnaire_results = pd.read_csv(fr'C:\Users\kevin\Desktop\ExperimentResults\VR Ausweichstudie Fragebogen.csv')
questionnaire_results.drop('Bitte geben Sie an, ob Sie die Datenschutz- und Einverständniserklärung gelesen haben und fragen Sie den Versuchsleiter, falls Ihnen diese nicht vorgelegt wurde, diese durchzusehen und zu unterzeichnen.', axis=1, inplace=True)
questionnaire_results['ID Nummer'] = questionnaire_results['ID Nummer'].str.upper()
# renaming IPQ columns:                 # general "sense of being there"
questionnaire_results.rename(columns={'In der computererzeugten Welt hatte ich den Eindruck, dort gewesen zu sein... ': 'G1',
                                        # Spatial Presence
                                      'Ich hatte das Gefühl, daß die virtuelle Umgebung hinter mir weitergeht.': 'SP1',
                                      'Ich hatte das Gefühl, nur Bilder zu sehen. ': 'SP2',
                                      'Ich hatte nicht das Gefühl, in dem virtuellen Raum zu sein': "SP3",
                                      'Ich hatte das Gefühl, in dem virtuellen Raum zu handeln statt etwas von außen zu bedienen.': 'SP4',
                                      'Ich fühlte mich im virtuellen Raum anwesend. ': 'SP5',
                                        # Involvement 
                                      'Wie bewusst war Ihnen die reale Welt, während Sie sich durch die virtuelle Welt bewegten (z.B. Geräusche, Raumtemperatur, andere Personen etc.)? ':'INV1',
                                      'Meine reale Umgebung war mir nicht mehr bewusst.': 'INV2',
                                      'Ich achtete noch auf die reale Umgebung. ': 'INV3',
                                      'Meine Aufmerksamkeit war von der virtuellen Welt völlig in Bann gezogen. ':'INV4',
                                        # Experienced Realism
                                      'Wie real erschien Ihnen die virtuelle Umgebung? ': 'REAL1',
                                      'Wie sehr glich Ihr Erleben der virtuellen Umgebung dem Erleben einer realen Umgebung? ': "REAL2",
                                      'Wie real erschien Ihnen die virtuelle Welt? ': 'REAL3',
                                      'Die virtuelle Welt erschien mir wirklicher als die reale Welt. ': 'REAL4',                                      
                                      }, inplace=True)
questionnaire_results['VRSQ-Score'] = [None] * len(questionnaire_results)
questionnaire_results['SP-Score'] = [None] * len(questionnaire_results)
questionnaire_results['INV-Score'] = [None] * len(questionnaire_results)
questionnaire_results['REAL-Score'] = [None] * len(questionnaire_results)

questionnaire_dict = {}
for i in range(len(questionnaire_results)):
        questionnaire_dict[questionnaire_results.loc[i]['ID Nummer']] = questionnaire_results.loc[i]

#questionnaire_results
questionnaire_dict['B7']

Zeitstempel                                                                                                                             2022/12/27 8:09:30 PM MEZ
ID Nummer                                                                                                                                                      B7
Allgemeines Unwohlsein                                                                                                                                          1
Ermüdung                                                                                                                                                        3
Kopfschmerzen                                                                                                                                                   2
                                                                                                                                                  ...            
In welchem Ausmaß spielen Si

***
### Prepare Data

Create a dictionary for each group A and B <br>
Ditctionary keys are the trial IDs<br>
Every key contains an array of all *MAX_YAW* samples from each user<br>

In [37]:
trial_id_list = []      # list of all trial IDs
for i in range(len(a_result_dict['A10'])):
    trial_id_list.append(a_result_dict['A10'].iloc[i]['trialID'])

a_yaws_by_trial = {}    # maximum accumulated virtual yaw per trial in grop A
b_yaws_by_trial = {}    # maximum accumulated real yaw per trial in grop B

for trial_id in trial_id_list:
    a_yaws_by_trial[f'{trial_id}'] = []
    b_yaws_by_trial[f'{trial_id}'] = []
    
    for user_id in a_result_dict:
        a_yaws_by_trial[f'{trial_id}'].append(a_result_dict[user_id].loc[a_result_dict[user_id]['trialID'] == trial_id]['max_yaw_virt'].values[0])
        #a_yaws_by_trial[f'{trial_id}'].append(abs(a_result_dict[user_id].loc[a_result_dict[user_id]['trialID'] == trial_id]['max_yaw_virt'].values[0]))

    for user_id in b_result_dict:
        b_yaws_by_trial[f'{trial_id}'].append(b_result_dict[user_id].loc[b_result_dict[user_id]['trialID'] == trial_id]['max_yaw_real'].values[0])
        #b_yaws_by_trial[f'{trial_id}'].append(abs(b_result_dict[user_id].loc[b_result_dict[user_id]['trialID'] == trial_id]['max_yaw_real'].values[0]))

In [33]:
a_yaws_by_trial['1026']

[114.8068, 36.1826, 110.3972, 93.69264]

### check max yaw samples of each trial for normal distribution

code taken from: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html </br>
see also: https://stackoverflow.com/questions/12838993/scipy-normaltest-how-is-it-used </br>
see also: https://www.pythonfordatascience.org/independent-samples-t-test-python/

In [25]:
all_samples_normally_distributed = True

print(f'H0: There are no significant differences in the REAL max yaw yalues of Group B and the VIRTUAL max yaw values of Group A for a specific trial.')

for trial_id in trial_id_list:
    a = a_yaws_by_trial[f'{trial_id}']
    b = b_yaws_by_trial[f'{trial_id}']
    x = np.concatenate((a, b))                                  # ???? Normalverteilung für Gruppe A und B zusammen oder getrennt testen ???

    k2, p_normal = stats.normaltest(x)
    alpha = 0.05                                                # ???? Kann / Soll man selbe Alpha für Normalverteilung und t-Test nutzen? In SciPy Docs Beispiel ist alpha 0.001 bzw. 1e-3 ???

    if p_normal < alpha:
        print(f'!!! Trial {trial_id} is not normally distributed !!! p is {p_normal}')        # ??? Kann man die Trials, die nicht normal verteilt sind, einfach ignorieren ???
        all_samples_normally_distributed = False
    else:
        _, p_ttest = stats.ttest_ind(a, b)
        if p_ttest < alpha:            
            print(f'H0 can be REJECTED for Trial {trial_id}')
        # else:
        #     print(f'H0 can NOT be rejected for Trial {trial_id}')

if all_samples_normally_distributed:
    print('All yaw samples are normally distributed.')

H0: There are no significant differences in the REAL max yaw yalues of Group B and the VIRTUAL max yaw values of Group A for a specific trial.
!!! Trial 23 is not normally distributed !!! p is 0.019891891945157707
H0 can be REJECTED for Trial 2006


c:\Users\kevin\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


!!! Trial 121 is not normally distributed !!! p is 0.001458789195172636
!!! Trial 2126 is not normally distributed !!! p is 0.03182007369602715
!!! Trial 1121 is not normally distributed !!! p is 0.0031547497975125413
!!! Trial 101 is not normally distributed !!! p is 0.04704966126713729
H0 can be REJECTED for Trial 1021
!!! Trial 104 is not normally distributed !!! p is 9.120370948492102e-05
!!! Trial 2124 is not normally distributed !!! p is 0.0005109021080308225
!!! Trial 1025 is not normally distributed !!! p is 1.6684703752884284e-05
!!! Trial 2122 is not normally distributed !!! p is 0.0364677441608688
!!! Trial 6 is not normally distributed !!! p is 0.014483807273905771
!!! Trial 114 is not normally distributed !!! p is 0.022314070037747782
!!! Trial 116 is not normally distributed !!! p is 0.037626843670153044
!!! Trial 10 is not normally distributed !!! p is 0.03527919297991528


In [36]:
print(f'H0: There are no significant differences in the REAL max yaw yalues of Group B and the VIRTUAL max yaw values of Group A for a specific trial.')

for trial_id in trial_id_list:
    a = pd.Series(a_yaws_by_trial[f'{trial_id}'])
    b = pd.Series(b_yaws_by_trial[f'{trial_id}'])

    _, p_ttest = stats.ttest_ind(a, b)

    if p_ttest < alpha:            
        print(f'H0 can be REJECTED for Trial {trial_id}')
        continue

    sampling_difference = a.values - b.values

    _, p_shapiro = stats.shapiro(sampling_difference)

    if p_shapiro < alpha:
        print(f'Trial {trial_id} is not normally distributed:')
        # fig = plt.figure(figsize= (20, 10))
        # ax = fig.add_subplot(111)

        # normality_plot, stat = stats.probplot(sampling_difference, plot= plt, rvalue= True)
        # ax.set_title("Probability plot of sampling difference", fontsize= 20)
        # ax.set

        # plt.show()
    else:
        _, p_levene = stats.levene(a, b, center= 'mean')

        if p_levene < alpha:
            print(f'Variances of group A and B in Trial {trial_id} are not equal.')
            # fig = plt.figure(figsize= (20, 10))
            # ax = fig.add_subplot(111)

            # p_bp_male = plt.hist(a, label= "Group A", density= True, alpha=0.75)
            # p_bp_female = plt.hist(b, label= "Group B", density= True, alpha=0.75)

            # plt.suptitle(f"Maximum Yaw in Trial {trial_id} \n between Group A and B", fontsize= 20)
            # plt.xlabel("Maximum Yaw", fontsize= 16)
            # plt.ylabel("Probability density", fontsize= 16)

            # plt.text(133, .025, f"$\mu= {b.mean(): .1f}, \ \sigma= {b.std(): .1f}$")
            # plt.text(160, .025, f"$\mu= {a.mean(): .1f}, \ \sigma= {a.std(): .1f}$")

            # plt.show()



H0: There are no significant differences in the REAL max yaw yalues of Group B and the VIRTUAL max yaw values of Group A for a specific trial.
Variances of group A and B in Trial 111 are not equal.
Variances of group A and B in Trial 120 are not equal.
Variances of group A and B in Trial 123 are not equal.
Variances of group A and B in Trial 2025 are not equal.
Trial 1106 is not normally distributed:
Trial 2012 is not normally distributed:
H0 can be REJECTED for Trial 2006
Trial 2024 is not normally distributed:
Trial 102 is not normally distributed:
Variances of group A and B in Trial 122 are not equal.
H0 can be REJECTED for Trial 1021
Variances of group A and B in Trial 1103 are not equal.
Variances of group A and B in Trial 1015 are not equal.
Variances of group A and B in Trial 1120 are not equal.
Variances of group A and B in Trial 1011 are not equal.
Variances of group A and B in Trial 1101 are not equal.
Trial 2124 is not normally distributed:
Trial 1025 is not normally distrib

***
### Calculate Questionnaire Data
#### VRSQ Score:

In [8]:
b_score, a_score = 0, 0
count_a, count_b = 0, 0

for key in questionnaire_dict:
    user_answers = questionnaire_dict[key]
    a = user_answers['Allgemeines Unwohlsein'] + user_answers['Ermüdung'] + user_answers['Kopfschmerzen'] + user_answers['angestrengte Augen']
    b = user_answers['Schwierigkeiten scharf zu sehen'] + user_answers['Kopfdruck'] + user_answers['Verschwommenes Sehen'] + user_answers['Schwindel bei geschlossenen Augen'] + user_answers['Gleichgewichtsstörungen']
    c = a / 12 * 100
    d = b / 15 * 100
    user_score = (c + d) / 2
    questionnaire_dict[key]['VRSQ-Score'] = user_score
    print('VRSQ Score for', user_answers['ID Nummer'], 'is', questionnaire_dict[key]['VRSQ-Score'])

    # sum up scores of all keys containing A and B
    if 'A' in key:
        a_score += user_score
        count_a += 1
    elif 'B' in key:
        b_score += user_score
        count_b += 1

# calculate average scores
a_score = a_score / count_a
b_score = b_score / count_b

print('Group A VRSQ-Score is ', a_score)
print('Group B VRSQ-Score is ', b_score)

VRSQ Score for A8 is 45.0
VRSQ Score for B4 is 33.33333333333333
VRSQ Score for A10 is 33.33333333333333
VRSQ Score for B5 is 74.16666666666666
VRSQ Score for A11 is 52.5
VRSQ Score for B6 is 36.666666666666664
VRSQ Score for A12 is 100.83333333333333
VRSQ Score for B7 is 60.0
VRSQ Score for A14 is 48.33333333333333
VRSQ Score for B9 is 51.66666666666667
VRSQ Score for A15 is 56.66666666666666
VRSQ Score for B11 is 63.33333333333333
VRSQ Score for A16 is 75.0
VRSQ Score for B12 is 66.66666666666666
VRSQ Score for A17 is 67.5
VRSQ Score for B13 is 54.166666666666664
Group A VRSQ-Score is  59.89583333333333
Group B VRSQ-Score is  54.99999999999999


C:\Users\kevin\AppData\Local\Temp\ipykernel_336\3472657793.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questionnaire_dict[key]['VRSQ-Score'] = user_score


#### IPQ Scores:

In [9]:
a_g, a_sp, a_inv, a_real = 0, 0, 0, 0
b_g, b_sp, b_inv, b_real = 0, 0, 0, 0
count_a, count_b = 0, 0

for key in questionnaire_dict:
    user_answers = questionnaire_dict[key]
    sp = (user_answers['SP1'] + user_answers['SP2'] + user_answers['SP3'] + user_answers['SP4'] + user_answers['SP5']) / 5
    inv = (user_answers['INV1'] + user_answers['INV2'] + user_answers['INV3'] + user_answers['INV4']) / 4
    real = (user_answers['REAL1'] + user_answers['REAL2'] + user_answers['REAL3'] + user_answers['REAL4']) / 4

    questionnaire_dict[key]['SP-Score'] = sp
    questionnaire_dict[key]['INV-Score'] = inv
    questionnaire_dict[key]['REAL-Score'] = real
    
    # print('IPQ Sub-Scores for', user_answers['ID Nummer'], 'are:')
    # print('General Presence: ', questionnaire_dict[key]['G1'])
    # print('Spatial Presence: ', questionnaire_dict[key]['SP-Score'])
    # print('Involvement: ', questionnaire_dict[key]['INV-Score'])
    # print('Realism: ', questionnaire_dict[key]['REAL-Score'])

    # sum up scores of all keys containing A and B
    if 'A' in key:
        a_g += questionnaire_dict[key]['G1']
        a_sp += sp
        a_inv += inv
        a_real += real
        count_a += 1
    elif 'B' in key:
        b_g += questionnaire_dict[key]['G1']
        b_sp += sp
        b_inv += inv
        b_real += real
        count_b += 1

# calculate average scores
a_g = a_g / count_a
a_sp = a_sp / count_a
a_inv = a_inv / count_a
a_real = a_real / count_a

b_g = b_g / count_b
b_sp = b_sp / count_b
b_inv = b_inv / count_b
b_real = b_real / count_b

print('IPQ Sub-Scores for Group A:')
print('General Presence: ', a_g)
print('Spatial Presence: ', a_sp)
print('Involvement: ', a_inv)
print('Realism: ', a_real)

print()

print('IPQ Sub-Scores for Group B:')
print('General Presence: ', b_g)
print('Spatial Presence: ', b_sp)
print('Involvement: ', b_inv)
print('Realism: ', b_real)

IPQ Sub-Scores for Group A:
General Presence:  4.5
Spatial Presence:  4.5249999999999995
Involvement:  4.1875
Realism:  4.4375

IPQ Sub-Scores for Group B:
General Presence:  5.375
Spatial Presence:  4.625
Involvement:  4.28125
Realism:  3.75


C:\Users\kevin\AppData\Local\Temp\ipykernel_336\652424499.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questionnaire_dict[key]['SP-Score'] = sp
C:\Users\kevin\AppData\Local\Temp\ipykernel_336\652424499.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questionnaire_dict[key]['INV-Score'] = inv
C:\Users\kevin\AppData\Local\Temp\ipykernel_336\652424499.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questionnaire_dict[key

***
### how to reference Pandas Dateframes

reference Samples

In [6]:
user_id = "A10"
trial_iteration = 25
sample = 2
sample_key = "g_r"

# get the whole dataframe of a trial
a_sample_dict[user_id][trial_iteration]

# get sample list:
a_sample_dict[user_id][trial_iteration][sample_key]

# get all values from specific sample
a_sample_dict[user_id][trial_iteration].iloc[sample]

# get a specific value 
a_sample_dict[user_id][trial_iteration].iloc[sample][sample_key]

# print all sample key names
a_sample_dict[user_id][trial_iteration].columns

Index(['sampling_intervals', 'g_r', 'injected_rotations',
       'real_rotation_accumulation',
       'additional_virtual_rotation_accumulation', 'real_position',
       'virtual_position', 'real_direction', 'virtual_direction', 'real_euler',
       'virtual_euler', 'real_rotation', 'virtual_rotation',
       'relative_virt_position', 'Unnamed: 14'],
      dtype='object')

reference Results

In [7]:
user_id = "A10"
trial_iteration = 25
trial = 80
column = "trialID"

# get the whole result dataframe of a trial
a_result_dict[user_id]
# get a column:
a_result_dict[user_id][column]
# get a row
a_result_dict[user_id].iloc[trial]
# get a specific value
a_result_dict[user_id].iloc[trial][column]
# print all column names
a_result_dict[user_id].columns

Index(['TrialIteration', 'EndState', 'experiment_start_time', 'trialID',
       'form', 'size', 'speed', 'angle', 'experiment_duration',
       'average_sampling_interval', 'sum_real_distance_travelled(IN METERS)',
       'max_distance_to_center', 'real_pos_at_max_dist',
       'virt_pos_at_max_dist', 'min_g_r', 'max_g_r', 'g_r_average',
       'sum_injected_rotation_g_r(IN DEGREES)', 'injected_rotation_average',
       'max_yaw_real', 'max_yaw_virt', 'real_dir_at_real_yaw',
       'virt_dir_at_real_yaw', 'real_dir_at_virt_yaw', 'virt_dir_at_virt_yaw',
       'real_pos_at_real_yaw', 'virt_pos_at_real_yaw', 'real_pos_at_virt_yaw',
       'virt_pos_at_virt_yaw', 'real_euler_at_real_yaw',
       'virt_euler_at_real_yaw', 'real_euler_at_virt_yaw',
       'virt_euler_at_virt_yaw', 'real_rot_at_real_yaw',
       'virt_rot_at_real_yaw', 'real_rot_at_virt_yaw', 'virt_rot_at_virt_yaw'],
      dtype='object')

split multidimensional samples with .split(",") like this:

In [14]:
x, y, z, w = a_result_dict[user_id].iloc[trial_iteration]['real_rot_at_real_yaw'].split(',')
print(x)
print(y)
print(z)
print(w)

KeyError: 'B7'

***

In [ ]:
def results_where_key_is_max(dict, user_id, key):
    max_id = dict[user_id].loc[:, key].idxmax()
    print("TrialID:", dict[user_id].loc[max_id, 'trialID'])
    print("Form:", dict[user_id].loc[max_id, 'form'])
    print("Size:", dict[user_id].loc[max_id, 'size'])
    print("Speed:", dict[user_id].loc[max_id, 'speed'])
    print("Angle:", dict[user_id].loc[max_id, 'angle'])
    print("min g/r:", dict[user_id].loc[max_id, 'min_g_r'])
    print("max g/r:", dict[user_id].loc[max_id, 'max_g_r'])
    print("g/r average:", dict[user_id].loc[max_id, 'g_r_average'])
    print("max Yaw:", dict[user_id].loc[max_id, 'max_yaw_real'])

    min_id = dict[user_id].loc[:, key].idxmin()
    if(dict[user_id].loc[max_id, key] < abs(dict[user_id].loc[min_id, key])):
        print()
        print("But absolute maximum is")
        results_where_key_is_min(dict ,user_id, key)

def results_where_key_is_min(dict, user_id, key):
    min_id = dict[user_id].loc[:, key].idxmin()
    print("TrialID:", dict[user_id].loc[min_id, 'trialID'])
    print("Form:", dict[user_id].loc[min_id, 'form'])
    print("Size:", dict[user_id].loc[min_id, 'size'])
    print("Speed:", dict[user_id].loc[min_id, 'speed'])
    print("Angle:", dict[user_id].loc[min_id, 'angle'])
    print("min g/r:", dict[user_id].loc[min_id, 'min_g_r'])
    print("max g/r:", dict[user_id].loc[min_id, 'max_g_r'])
    print("g/r average:", dict[user_id].loc[min_id, 'g_r_average'])
    print("max Yaw:", dict[user_id].loc[min_id, 'max_yaw_real'])

# create a numpy array of all the values in the column

In [ ]:
results_where_key_is_max(b_result_dict, 'B4', 'max_yaw_real')

TrialID: 14
Form: Sphere
Size: 0.3
Speed: 24
Angle: -13
min g/r: nan
max g/r: nan
g/r average: 0
max Yaw: 177.2879

But absolute biggest is
TrialID: 101
Form: Sphere
Size: 0.8
Speed: 18
Angle: 27
min g/r: nan
max g/r: nan
g/r average: 0
max Yaw: -222.0172
